In [1]:
import requests
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import time
import re
import os
import sys

In [2]:
link='https://stock-pictures.netlify.app/'
driver = webdriver.Chrome()

driver.get(link)

time.sleep(10)
soup=BeautifulSoup(driver.page_source, 'html.parser')



In [3]:
print(soup.prettify())

<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <link href="/favicon.ico" rel="icon"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="#000000" name="theme-color"/>
  <meta content="Web site created using create-react-app" name="description"/>
  <link href="/logo192.png" rel="apple-touch-icon"/>
  <link href="/manifest.json" rel="manifest"/>
  <title>
   Stock Images
  </title>
  <script defer="defer" src="/static/js/main.67034cee.js">
  </script>
  <link href="/static/css/main.07ec7845.css" rel="stylesheet"/>
 </head>
 <body>
  <noscript>
   You need to enable JavaScript to run this app.
  </noscript>
  <div id="root">
   <div class="App">
    <h1>
     Stock Images with Tags
    </h1>
    <div class="infinite-scroll-component__outerdiv">
     <div class="infinite-scroll-component" style="height: auto; overflow: auto;">
      <div class="main">
       <div class="container">
        <div class="image-container">
         <img alt="src" clas

In [4]:
data=[]
for sp in soup.find_all('div',class_='container'):
    img_link=sp.find('img',class_='source-img').get('src')
    tags=sp.find('span',class_='tag-color').text[7:].split(',')
    likes=int(re.sub(r'[^\d]','',sp.find('div',class_='likes-comments').find_all('span')[0].text))
    comments=int(re.sub(r'[^\d]','',sp.find('div',class_='likes-comments').find_all('span')[1].text))
    data.append([img_link,tags,likes,comments])

In [9]:
print(len(data))
df=pd.DataFrame(data,columns=['img_link','tags','likes','comments'])

48


In [7]:
path_list=[]
for link in df['img_link']:
    fd=open(f'scraped-images/{link.split("/")[-1]}','wb')
    fd.write(requests.get(link).content)
    fd.close()    
    path=f'/scraped-images/{link.split("/")[-1]}'
    path_list.append(path)

In [10]:
df['path']=path_list
df.head()

,img_link,tags,likes,comments,path
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"[Clouds, Sky, Atmosphere, Blue Sky]",196,55,/scraped-images/clouds-7050884__480.jpg
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"[Bird, Ornithology, Hummingbird]",76,20,/scraped-images/bird-7117346__340.jpg
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"[Sea, Rainbow, Rainfall, Subtropical]",282,106,/scraped-images/sea-7039471__340.jpg
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"[Cherry Blossoms, Road, Japan, Sakura]",42,11,/scraped-images/cherry-blossoms-7110279__340.jpg
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"[Cape Marguerite, Flower, Plant]",39,15,/scraped-images/cape-marguerite-7121992__340.jpg


In [11]:
df.to_csv('scraped-data.csv',index=False)